# Extrator de Golos e Marcadores - SC Farense

Extrai todos os golos e marcadores de TODOS os jogos do SC Farense do ZeroZero.pt

**Fonte**: https://www.zerozero.pt

**Saída**: JSON com estrutura:
```json
{
  "jogo_id": "123456",
  "data": "2023-01-15",
  "equipas": {"casa": "SC Farense", "fora": "CD Aves"},
  "resultado": {"casa": 2, "fora": 1},
  "golos": [
    {"minuto": "15", "marcador": "João Silva", "equipa": "Farense", "assistência": "Carlos"}
  ]
}
```

## 1. Importar Bibliotecas

In [ ]:
import asyncio
import aiohttp
import json
import re
from bs4 import BeautifulSoup
from typing import List, Dict, Optional
from datetime import datetime
import logging
from pathlib import Path

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Bibliotecas importadas com sucesso")

## 2. Classe Principal - Extrator de Golos

In [ ]:
class GolosExtrator:
    """Extrai golos e marcadores de jogos do Farense no ZeroZero"""
    
    BASE_URL = "https://www.zerozero.pt"
    FARENSE_URLS = [
        # URLs de jogos do Farense (será populado dinamicamente)
    ]
    
    def __init__(self, max_concurrent: int = 5, timeout: int = 15):
        self.max_concurrent = max_concurrent
        self.timeout = timeout
        self.session = None
        self.golos_por_jogo = []
        self.erros = []
        
    async def init_session(self):
        """Inicializa sessão aiohttp"""
        self.session = aiohttp.ClientSession(
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml',
                'Accept-Language': 'pt-PT,pt;q=0.9',
            },
            timeout=aiohttp.ClientTimeout(total=self.timeout)
        )
        logger.info("✓ Sessão HTTP inicializada")
    
    async def close_session(self):
        """Fecha sessão aiohttp"""
        if self.session:
            await self.session.close()
            logger.info("✓ Sessão HTTP fechada")
    
    async def fetch_page(self, url: str) -> Optional[str]:
        """Fetch página com retry"""
        for tentativa in range(3):
            try:
                async with self.session.get(url) as response:
                    if response.status == 200:
                        return await response.text()
                    else:
                        logger.warning(f"Status {response.status}: {url}")
            except asyncio.TimeoutError:
                logger.warning(f"Timeout em {url} (tentativa {tentativa+1}/3)")
            except Exception as e:
                logger.warning(f"Erro em {url}: {e}")
            
            if tentativa < 2:
                await asyncio.sleep(2 ** tentativa)
        
        self.erros.append(url)
        return None
    
    def extrair_golos_da_pagina(self, html: str, jogo_id: str) -> Dict:
        """Extrai golos de uma página de jogo"""
        try:
            soup = BeautifulSoup(html, 'html.parser')

            # Extrair informações do jogo (equipas, resultado)
            jogo_info = self._extrair_info_jogo(soup)

            # Extrair golos - múltiplas estratégias
            golos = []

            # Estratégia 1: Procurar divs com classe 'goal' ou 'scorer'
            goal_elements = soup.find_all(['div', 'span'], class_=re.compile('goal|golo|scorer|marcador|events?'))

            if not goal_elements:
                # Estratégia 2: Procurar na timeline/events section
                timeline = soup.find(['div', 'section'], class_=re.compile('timeline|events|match.*events|match.*info'))
                if timeline:
                    goal_elements = timeline.find_all(['div', 'li', 'span'])

            # Processar elementos encontrados
            for elem in goal_elements:
                golo = self._extrair_golo_do_elemento(elem, jogo_info)
                if golo:
                    golos.append(golo)

            # Estratégia 3: Se ainda não encontrou, procurar por padrões de texto
            if not golos:
                golos = self._extrair_golos_por_regex(html, jogo_info)

            return {
                'jogo_id': jogo_id,
                'data_extracao': datetime.now().isoformat(),
                'jogo_info': jogo_info,
                'golos': golos,
                'total_golos': len(golos)
            }

        except Exception as e:
            logger.error(f"Erro ao processar jogo {jogo_id}: {e}")
            return None

    def _extrair_info_jogo(self, soup) -> Dict:
        """Extrai informações básicas do jogo"""
        jogo_info = {
            'equipas': {'casa': None, 'fora': None},
            'resultado': {'casa': None, 'fora': None},
            'data': None
        }

        try:
            # Procurar score/equipas
            score_section = soup.find(['div', 'section'], class_=re.compile('score|match.*info|game.*info'))
            if score_section:
                # Extrair nomes das equipas
                team_names = score_section.find_all(['h1', 'h2', 'h3', 'span', 'a'])
                if len(team_names) >= 2:
                    jogo_info['equipas']['casa'] = team_names[0].get_text(strip=True)
                    jogo_info['equipas']['fora'] = team_names[-1].get_text(strip=True)

                # Extrair resultado
                score_match = re.search(r'(\d+)\s*[-:]\s*(\d+)', score_section.get_text())
                if score_match:
                    jogo_info['resultado']['casa'] = int(score_match.group(1))
                    jogo_info['resultado']['fora'] = int(score_match.group(2))

            # Extrair data
            date_elem = soup.find(text=re.compile(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2}'))
            if date_elem:
                jogo_info['data'] = str(date_elem).strip()

        except Exception as e:
            logger.debug(f"Erro ao extrair info do jogo: {e}")

        return jogo_info

    def _extrair_golo_do_elemento(self, elem, jogo_info) -> Optional[Dict]:
        """Extrai golo de um elemento HTML"""
        try:
            texto = elem.get_text(strip=True)

            # Procurar padrão de minuto
            minuto_match = re.search(r"(\d+)\s*['\+]\s*(\d*)", texto)
            if not minuto_match:
                return None

            minuto = minuto_match.group(1)

            # Procurar nome do marcador (link ou texto próximo)
            marcador = None
            link = elem.find('a')
            if link:
                marcador = link.get_text(strip=True)
            else:
                # Tentar extrair nome após minuto
                nome_match = re.search(r"\d+['\+]\s*(.+?)(?:\(|,|$)", texto)
                if nome_match:
                    marcador = nome_match.group(1).strip()

            if not marcador or marcador.lower() in ['goal', 'golo', 'o.g.']:
                return None

            # Determinar equipa
            equipa = self._determinar_equipa(texto, elem, jogo_info)

            # Extrair assistência
            assistencia = self._extrair_assistencia(texto)

            return {
                'minuto': minuto,
                'marcador': marcador,
                'equipa': equipa,
                'assistencia': assistencia
            }

        except Exception as e:
            logger.debug(f"Erro ao extrair golo: {e}")
            return None

    def _determinar_equipa(self, texto: str, elem, jogo_info: Dict) -> str:
        """Determina qual equipa marcou o golo"""
        # Verificar contexto visual (classe do elemento)
        classes = elem.get('class', [])

        for cls in classes:
            if 'home' in cls.lower() or 'casa' in cls.lower():
                return jogo_info['equipas']['casa'] or 'Casa'
            elif 'away' in cls.lower() or 'fora' in cls.lower():
                return jogo_info['equipas']['fora'] or 'Fora'

        # Verificar por padrões no texto (ex: OG, Autogolo)
        if any(x in texto.lower() for x in ['og', 'autogolo', 'own goal']):
            return 'Autogolo'

        # Default (Farense ou primeira equipa)
        return jogo_info['equipas']['casa'] or 'Farense'

    def _extrair_assistencia(self, texto: str) -> Optional[str]:
        """Extrai nome do assistidor"""
        # Procurar por padrões comuns: "assist: Nome", "(assist: Nome)"
        assist_match = re.search(r'(?:assist\.?|assist de|assist|assistência)[\s:]*([A-ZÁÉÍÓÚà-ÿ][a-zà-ÿ]+(?:\s+[A-ZÁÉÍÓÚà-ÿ][a-zà-ÿ]+)?)', texto, re.IGNORECASE)
        if assist_match:
            return assist_match.group(1).strip()
        return None

    def _extrair_golos_por_regex(self, html: str, jogo_info: Dict) -> List[Dict]:
        """Fallback: Extrai golos usando regex se parsing não funcionou"""
        golos = []

        # Padrão para encontrar golos: "minuto' Nome" ou "minuto+tempo Nome"
        golo_pattern = r"(\d+)[\s\+](\d*)\s*['\´]\s*([A-ZÁÉÍÓÚà-ÿ][a-zà-ÿ]+(?:\s+[A-ZÁÉÍÓÚà-ÿ][a-zà-ÿ]+)?)"

        matches = re.finditer(golo_pattern, html)
        for match in matches:
            minuto = match.group(1)
            marcador = match.group(3).strip()

            if marcador.lower() not in ['goal', 'golo', 'time', 'date', 'equipa']:
                golos.append({
                    'minuto': minuto,
                    'marcador': marcador,
                    'equipa': jogo_info['equipas']['casa'] or 'Farense',
                    'assistencia': None
                })

        return golos
    
    async def processar_jogo(self, url: str, jogo_id: str, indice: int, total: int):
        """Processa um jogo específico"""
        logger.info(f"[{indice}/{total}] Processando jogo {jogo_id}...")
        
        html = await self.fetch_page(url)
        if html:
            resultado = self.extrair_golos_da_pagina(html, jogo_id)
            if resultado:
                self.golos_por_jogo.append(resultado)
                logger.info(f"✓ Jogo {jogo_id}: {resultado['total_golos']} golos encontrados")
        
        await asyncio.sleep(0.5)  # Rate limiting
    
    async def processar_multiplos_jogos(self, urls: List[tuple]):
        """Processa múltiplos jogos com concorrência"""
        total = len(urls)
        semaphore = asyncio.Semaphore(self.max_concurrent)
        
        async def bounded_task(url, jogo_id, indice):
            async with semaphore:
                await self.processar_jogo(url, jogo_id, indice, total)
        
        tasks = [
            bounded_task(url, jogo_id, i+1)
            for i, (url, jogo_id) in enumerate(urls)
        ]
        
        await asyncio.gather(*tasks)
    
    def salvar_json(self, arquivo: str = "golos_farense.json"):
        """Salva golos em arquivo JSON"""
        with open(arquivo, 'w', encoding='utf-8') as f:
            json.dump(self.golos_por_jogo, f, ensure_ascii=False, indent=2)
        logger.info(f"✓ Dados salvos em {arquivo}")

print("✓ Classe GolosExtrator criada (versão melhorada)")

## 3. Função para Encontrar Links de Jogos do Farense

In [ ]:
async def encontrar_jogos_farense(extrator: GolosExtrator) -> List[tuple]:
    """
    Encontra todos os links de jogos do Farense.
    Retorna lista de tuplas (url, jogo_id)
    """
    
    urls_jogos = []
    
    # URLs base para procurar jogos do Farense
    search_urls = [
        "https://www.zerozero.pt/equipa/sc-farense/jogos",
        "https://www.zerozero.pt/equipa/sc-farense/jogos/",
    ]
    
    for search_url in search_urls:
        logger.info(f"Procurando jogos em {search_url}...")
        
        html = await extrator.fetch_page(search_url)
        if not html:
            continue
        
        soup = BeautifulSoup(html, 'html.parser')
        
        # Procurar links de jogos
        jogo_links = soup.find_all('a', href=re.compile(r'/jogo\.php\?id=\d+|/match/\d+'))
        
        for link in jogo_links:
            href = link.get('href')
            if href:
                # Extrair jogo_id
                id_match = re.search(r'id=(\d+)|match/(\d+)', href)
                if id_match:
                    jogo_id = id_match.group(1) or id_match.group(2)
                    
                    # Construir URL completa
                    if href.startswith('http'):
                        full_url = href
                    else:
                        full_url = f"{extrator.BASE_URL}{href}"
                    
                    urls_jogos.append((full_url, jogo_id))
        
        logger.info(f"Encontrados {len(jogo_links)} links de jogos")
        break  # Se encontrou links, não precisa tentar outros URLs
    
    return urls_jogos

print("✓ Função encontrar_jogos_farense criada")

## 4. Executar Extração - OPÇÃO A: URLs Manuais (Rápido)

In [ ]:
# OPÇÃO A: Usar URLs conhecidas (mais rápido para teste)
# Exemplos de jogos do Farense no ZeroZero:
jogos_urls = [
    # Jogo 1: Farense 2025
    # ("https://www.zerozero.pt/jogo.php?id=XXXXX", "XXXXX"),
    
    # Você pode encontrar URLs de jogos do Farense em:
    # https://www.zerozero.pt/equipa/sc-farense/
    # https://www.zerozero.pt/equipa/sc-farense/2024-2025/
]

print(f"URLs preparadas: {len(jogos_urls)}")
if len(jogos_urls) == 0:
    print("\n📝 Como usar OPÇÃO A:")
    print("1. Visite https://www.zerozero.pt/equipa/sc-farense/")
    print("2. Clique em um jogo específico")
    print("3. Copie a URL completa (ex: https://www.zerozero.pt/jogo.php?id=12345)")
    print("4. Adicione como tupla (url, jogo_id) na lista jogos_urls acima")
    print("\n⚠️  Ou use a OPÇÃO B abaixo para encontrar automaticamente.")

## 5. Executar Extração - OPÇÃO B: Auto-descoberta (Mais Lento)

In [ ]:
async def executar_extracao(urls_jogos: List[tuple]):
    """Executa a extração completa de golos do Farense"""
    
    if not urls_jogos:
        print("\n❌ Nenhuma URL fornecida!")
        print("\nUse OPÇÃO A ou OPÇÃO B para preparar as URLs primeiro.")
        return
    
    extrator = GolosExtrator(max_concurrent=3, timeout=20)
    
    try:
        # Inicializar sessão
        await extrator.init_session()
        
        print(f"\n=== INICIANDO EXTRAÇÃO ===")
        print(f"Total de jogos a processar: {len(urls_jogos)}")
        print(f"Max concorrência: {extrator.max_concurrent}")
        
        # Processar jogos
        await extrator.processar_multiplos_jogos(urls_jogos)
        
        # Salvar resultados
        print("\n=== SALVANDO RESULTADOS ===")
        extrator.salvar_json("golos_farense.json")
        
        # Estatísticas finais
        print(f"\n=== RESUMO FINAL ===")
        print(f"Jogos processados com sucesso: {len(extrator.golos_por_jogo)}")
        print(f"Falhas/Erros: {len(extrator.erros)}")
        
        if extrator.golos_por_jogo:
            total_golos = sum(j['total_golos'] for j in extrator.golos_por_jogo)
            print(f"Total de golos encontrados: {total_golos}")
            print(f"Média de golos por jogo: {total_golos / len(extrator.golos_por_jogo):.1f}")
    
    except Exception as e:
        logger.error(f"Erro na execução: {e}")
        print(f"\n❌ Erro: {e}")
    
    finally:
        await extrator.close_session()

# Executar com as URLs preparadas acima
print("Iniciando extração...\n")
await executar_extracao(jogos_urls)

## 6. Visualizar Resultados

In [ ]:
import json

# Carregar dados salvos
try:
    with open('golos_farense.json', 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    print(f"=== RESULTADOS ===")
    print(f"Total de jogos: {len(dados)}")
    print(f"\nPrimeiros 3 jogos:\n")
    
    for i, jogo in enumerate(dados[:3]):
        print(f"Jogo {i+1}:")
        print(f"  ID: {jogo['jogo_id']}")
        print(f"  Golos: {jogo['total_golos']}")
        for golo in jogo['golos']:
            assist = f" (assist: {golo['assistencia']})" if golo['assistencia'] else ""
            print(f"    - {golo['minuto']}' {golo['marcador']} ({golo['equipa']}){assist}")
        print()
except FileNotFoundError:
    print("Arquivo golos_farense.json não encontrado. Execute a extração primeiro.")

## 7. Análise e Estatísticas

In [ ]:
import pandas as pd
from collections import Counter

try:
    with open('golos_farense.json', 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    # Criar DataFrame
    todos_golos = []
    for jogo in dados:
        for golo in jogo['golos']:
            todos_golos.append({
                'jogo_id': jogo['jogo_id'],
                'minuto': int(golo['minuto']) if golo['minuto'].isdigit() else 0,
                'marcador': golo['marcador'],
                'equipa': golo['equipa'],
                'assistencia': golo['assistencia']
            })
    
    if todos_golos:
        df = pd.DataFrame(todos_golos)
        
        print("=== ANÁLISE DE GOLOS ===")
        print(f"\nTotal de golos: {len(df)}")
        print(f"Golos Farense: {len(df[df['equipa']=='Farense'])}")
        print(f"Golos Adversários: {len(df[df['equipa']!='Farense'])}")
        
        # Top marcadores
        marcadores_farense = df[df['equipa']=='Farense']['marcador'].value_counts()
        print(f"\n=== TOP 10 MARCADORES (FARENSE) ===")
        for marcador, gols in marcadores_farense.head(10).items():
            print(f"{marcador}: {gols} golo(s)")
        
        # Distribuição por minuto
        print(f"\n=== DISTRIBUIÇÃO POR MINUTO ===")
        print(f"Minutos mais frequentes:")
        print(df['minuto'].value_counts().head(10))
    else:
        print("Nenhum golo encontrado nos dados.")

except FileNotFoundError:
    print("Arquivo não encontrado.")
except Exception as e:
    print(f"Erro na análise: {e}")

## 8. Exportar para Diferentes Formatos

In [ ]:
def exportar_formatos(arquivo_json: str = 'golos_farense.json'):
    """Exporta dados para diferentes formatos"""
    
    try:
        with open(arquivo_json, 'r', encoding='utf-8') as f:
            dados = json.load(f)
        
        # Preparar dados para CSV
        todos_golos = []
        for jogo in dados:
            for golo in jogo['golos']:
                todos_golos.append({
                    'jogo_id': jogo['jogo_id'],
                    'data_extracao': jogo['data_extracao'],
                    'minuto': golo['minuto'],
                    'marcador': golo['marcador'],
                    'equipa': golo['equipa'],
                    'assistencia': golo['assistencia'] or ''
                })
        
        # Exportar CSV
        df = pd.DataFrame(todos_golos)
        csv_arquivo = 'golos_farense.csv'
        df.to_csv(csv_arquivo, index=False, encoding='utf-8')
        print(f"✓ Exportado para {csv_arquivo}")
        
        # Exportar Excel (se disponível)
        try:
            excel_arquivo = 'golos_farense.xlsx'
            df.to_excel(excel_arquivo, index=False, sheet_name='Golos')
            print(f"✓ Exportado para {excel_arquivo}")
        except ImportError:
            print("⚠️  openpyxl não instalado. Pulando Excel.")
    
    except FileNotFoundError:
        print(f"Arquivo {arquivo_json} não encontrado")
    except Exception as e:
        print(f"Erro na exportação: {e}")

print("✓ Função exportar_formatos criada")
# Descomentar para exportar
# exportar_formatos()